In [1]:
# (1) パッケージをインストールする（初回のみ）
# !pip install sklearn

In [2]:
# (2) ライブラリの読み出し
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

In [3]:
# (3) データの読み込み
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

In [4]:
# (4) 女性向けメールが配信されたデータを削除したデータを作成
male_df = email_data.query('segment != "Womens E-Mail"').reset_index(drop=True)
male_df['treatment'] = (male_df['segment'] == "Mens E-Mail").astype(int)

In [5]:
# (5) セレクションバイアスのあるデータを作成
## seedを固定
random.seed(0)

## 条件に反応するサンプルの量を半分にする
obs_rate_c = 0.5
obs_rate_t = 0.5

## バイアスのあるデータの作成
male_df['obs_rate_c'] = np.where((male_df['history'] > 300) |  (male_df['recency'] < 6) | (male_df['channel']== 'Multichannel'),
                                                             obs_rate_c, 1)
male_df['obs_rate_t'] = np.where((male_df['history'] > 300) |  (male_df['recency'] < 6) | (male_df['channel']== 'Multichannel'),
                                                             1, obs_rate_t)
male_df['random_number'] = [random.random() for _ in range(len(male_df))]
biased_data = male_df.query('(treatment == 0 and random_number < obs_rate_c) or (treatment == 1 and random_number < obs_rate_t)').reset_index(drop=True)

In [6]:
# (6) バイアスのあるデータでの回帰分析
## 回帰分析の実行
biased_reg = LinearRegression().fit(biased_data[['treatment', 'history']], biased_data['spend'])
biased_reg.coef_

array([0.83272547, 0.00145485])

In [7]:
# (7) RCTデータでの回帰分析とバイアスのあるデータでの回帰分析の比較
## RCTデータでの単回帰
rct_reg = LinearRegression().fit(male_df[['treatment']], male_df['spend'])
rct_reg.coef_

array([0.76982716])

In [8]:
## バイアスのあるデータでの単回帰
nonrct_reg = LinearRegression().fit(biased_data[['treatment']], biased_data['spend'])
nonrct_reg.coef_

array([0.93786193])

In [9]:
## バイアスのあるデータでの重回帰
biased_data['channel'] = biased_data['channel'].map({'Web': 0, 'Phone': 1, 'Multichannel': 2})
nonrct_mreg = LinearRegression().fit(biased_data[['treatment', 'recency', 'channel', 'history']], biased_data['spend'])
nonrct_mreg.coef_

array([ 0.78399264, -0.03886176, -0.21631642,  0.00150815])

In [10]:
# (8) OVBの確認
## (a) history抜きの回帰分析とパラメーターの取り出し
alpha_1 = LinearRegression().fit(biased_data[['treatment', 'recency', 'channel']], biased_data['spend']).coef_[0]
beta_1 = LinearRegression().fit(biased_data[['treatment', 'recency', 'channel', 'history']], biased_data['spend']).coef_[0]
beta_2 = LinearRegression().fit(biased_data[['treatment', 'recency', 'channel', 'history']], biased_data['spend']).coef_[3]
gamma_1 = LinearRegression().fit(biased_data[['treatment', 'recency', 'channel']], biased_data['history']).coef_[0]

In [11]:
beta_2*gamma_1

0.05758160277479463

In [12]:
alpha_1 - beta_1

0.057581602774794294

In [13]:
# (10) 入れてはいけない変数を入れてみる
#visitとtreatmentとの相関
cor_visit_treatment = LinearRegression().fit(biased_data[['visit', 'channel', 'recency', 'history']], biased_data['treatment'])
cor_visit_treatment.coef_

array([ 1.49314279e-01,  2.18560484e-02, -2.90465912e-02,  1.42460758e-04])

In [14]:
# visitを入れた回帰分析を実行
bad_control_reg = LinearRegression().fit(biased_data[['treatment', 'channel', 'recency', 'history', 'visit']], biased_data['spend'])
bad_control_reg.coef_

array([ 1.88836771e-01, -8.13474538e-02,  9.41359577e-03,  9.10288118e-04,
        7.40794836e+00])